In [11]:
import pickle
import numpy as np
from pathlib import Path

import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaModel 
from torch.utils.data import Dataset, DataLoader

from dataset import HumorDataset

from pickle_loader import load_pickle
from typing import Tuple, List, Dict

device = 0 if torch.cuda.is_available() else -1
print(device)
model_name = 'Reggie/muppet-roberta-base-joke_detector'

0


In [6]:
dataset = HumorDataset()

In [7]:
sample = dataset[0]
sample

(1,
 "the mathematics of quantum mechanics very accurately describes how our universe works. and it tells us our reality is continually branching into different possibilities just like a coral. it's a weird thing for us humans to wrap our minds around since we only ever get to experience one possibility. this quantum weirdness was first described by erwin and his cat. the cat likes this version better",
 WindowsPath('../data/urfunny2_video/1.mp4'),
 WindowsPath('../data/urfunny2_audio/1.mp3'),
 1)

In [10]:
tokenizer = RobertaTokenizer.from_pretrained(model_name, model_max_length=510)
model = RobertaModel.from_pretrained(model_name).to(device)

input_seq = tokenizer(sample[1], '', truncation=True, return_tensors="pt")
output = model(input_seq["input_ids"].to(device))
prediction = torch.softmax(output["logits"][0], -1).tolist()
print(prediction)
pred_out = 1 if prediction[0] < prediction[1] else 0
print('This is a joke') if pred_out == 1 else print('This is not a joke')

None


TypeError: get_output_embeddings() takes 1 positional argument but 2 were given

False